In [180]:
import pyedflib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import scipy.signal

In [2]:
t_start = datetime.datetime.now()
f = pyedflib.EdfReader(r"D:\BaiduNetdiskDownload\sz1.edf")
t_end = datetime.datetime.now()
print(t_end-t_start)

0:00:00.002000


In [30]:
t_start = datetime.datetime.now()
n = f.signals_in_file
record = np.zeros((n, f.getNSamples()[0]))
for i in np.arange(n):
    record[i, :] = f.readSignal(i)
t_end = datetime.datetime.now()
print(t_end-t_start)

0:00:38.111001


In [80]:
def h2(x,y,L=7):
    '''
    x,y 代表两个信号
    L代表区间线
    该函数返回每个区间的Q、P值
    '''
    t_start_all = datetime.datetime.now()
    min_x = np.min(x)
    max_x = np.max(x)
    bins=np.linspace(min_x,max_x,L)
    #print(bins)
    sxy=pd.DataFrame({'x':x,'y':y})
    #sxy=sxy.sort_values(by='x')
    #sxy.index=range(len(sxy))
    
    sxy['bins']=pd.cut(sxy['x'],bins,labels=range(1,L),include_lowest=True)
    Q=[]
    P=[] # 6
    k=[] # 5
    f=[]
    for i in range(L-1):
        Q.append(sxy[sxy.bins==i+1]['y'].mean())
        P.append((bins[i]+bins[i+1])/2) 
        if i:
            k.append((Q[i]-Q[i-1])/(P[i]-P[i-1]))
    t_end = datetime.datetime.now()
    print("Q_P_k：",t_end-t_start_all)
    
    t_start = datetime.datetime.now()
    px=P.copy()
    px[0]=min_x
    px[L-2]=max_x
    #print(px)
    sxy['k']=pd.cut(sxy['x'],px,labels=range(0,L-2),include_lowest=True)
    t_end = datetime.datetime.now()
    print("切割：",t_end-t_start)
    
    t_start = datetime.datetime.now()
    for si in range(len(x)):
        i = sxy.loc[si]['k']
        fx=k[i]*(sxy.loc[si]['x']-P[i])+Q[i]
        f.append(fx)
    t_end = datetime.datetime.now()
    print("计算f：",t_end-t_start)
    
    t_start = datetime.datetime.now()
    my=np.mean(y)
    sst=np.sum((y-my)**2)
    sse=np.sum((y-f)**2)
    h2=1-sse/sst
    t_end = datetime.datetime.now()
    print("计算h2：",t_end-t_start)
    
    return h2

In [158]:
def h2_modify(x,y,L=7):
    '''
    x,y 代表两个信号
    L代表区间线
    该函数返回每个区间的Q、P值
    '''
    min_x = np.min(x)-0.00000001
    max_x = np.max(x)
    lx = len(x)
    bins=np.linspace(min_x,max_x,L)  
    Q=[]
    P=[] # 6
    k=[] # 5
    f=[]
    Qy=[None]*(L-1)
    for i in range(L-1):
        Qy[i] = []
        
    for xi in range(lx):
        for li in range(L-1):
            if bins[li] < x[xi] <=bins[li+1]:
                Qy[li].append(y[xi])
                break
    
    for i in range(L-1):
        Q.append(np.mean(Qy[i]))
        P.append((bins[i]+bins[i+1])/2) 
        if i:
            k.append((Q[i]-Q[i-1])/(P[i]-P[i-1]))

    px=P.copy()
    px[0]=min_x
    px[L-2]=max_x
    
    for xi in x:
        for li in range(L-2):
            if px[li] < xi <= px[li+1]:
                fx=k[li]*(xi-P[li])+Q[li]
                f.append(fx)
                break
  
    #f = np.array(f) 当长度相等，可以不加
    my=np.mean(y)
    sst=np.sum((y-my)**2)
    sse=np.sum((y-f)**2)
    h2=1-sse/sst   
    return h2

In [189]:
def H2_filter(signals,start_time,duration,sampleFreq,slideWindow=2,step=1,maxlag=0.1,L=7,HP=0,LP=0):
    '''
    在H2_whole3方法的基础上增加filter功能
    filter为长度为2的数组，接收归一化截止频率：HP，LP
    '''
    #time_flexity=1
    signals_num = len(signals)
    if signals_num < 2:
        print("Please input at least 2 signals to compute h2!")
        return None,None
    
    h2_num = int((duration-slideWindow-maxlag)//step + 1)
    if h2_num <= 0:
        print("Duration is too short to compute h2!")
        return None,None
    
    if maxlag <= 0:
        print("Time maxlag must be greater than zero!")
        
    b = a = []
    if HP==0 and LP>0: #低通
        b,a = scipy.signal.butter(2,2*LP/sampleFreq,"lowpass")
    elif LP==0 and HP>0: #高通
        b,a = scipy.signal.butter(2,2*HP/sampleFreq,"highpass")
    elif LP>0 and HP>0: #带通
        b,a = scipy.signal.butter(2,[2*HP/sampleFreq,2*LP/sampleFreq],"bandpass")
    
    h2_value = []
    lag_value = []
    
     # 计算每次窗口滑动时的H2（每个时间段）
    for ti in range(h2_num):
        start_time_window = start_time + ti*step
        stop_time_window = start_time_window + slideWindow
        start_samples = int(start_time_window*sampleFreq)
        stop_samples = int(stop_time_window*sampleFreq)
        # 对角线上的值设置为 0
        signals_h2 = np.zeros((signals_num,signals_num))
        signals_lag = np.zeros((signals_num,signals_num))
        # 计算每两个信号之间的H2: (0,1)，(0,2)，...，(0,n-1)；(1,2)，(1,3)，...，(1,n-1)；...(n-2,n-1)
        for si1 in range(signals_num-1):
            s1 = signals[si1][start_samples:stop_samples]
            if len(a):
                s1 = scipy.signal.filtfilt(b,a,s1)
            # [s1] = setSignaltoZero([signals[si1][start_samples:stop_samples]]) # 做归0处理
            for si2 in range(si1+1,signals_num):
                s2 = signals[si2][start_samples:stop_samples]
                if len(a):
                    s2 = scipy.signal.filtfilt(b,a,s2)
                #[s2] = setSignaltoZero([signals[si2][start_samples:stop_samples]])
                # 设置两个信号之间的H2的初始值为0
                two_signals_h2 = np.zeros(2) 
                two_signals_lag = np.zeros(2) 
                
                h2_00 = [h2_modify(s1,s2,L),h2_modify(s2,s1,L)]
                #time_flexity += 1
                two_signals_h2[0] = h2_00[0]
                two_signals_h2[1] = h2_00[1]
                # 考虑时间延迟
                # 计算x到y
                sample_maxlag = int(maxlag*sampleFreq)
                for tagi in range(1,sample_maxlag): # x向右移动，符号为正；y向右移动，符号为负
                    #print(start_samples,stop_samples)
                    #time_flexity += 2
                    s1m = signals[si1][start_samples+tagi:stop_samples+tagi]
                    s2m = signals[si2][start_samples+tagi:stop_samples+tagi]
                    #[s1m,s2m] = setSignaltoZero([s1m,s2m],1)  # 做归0处理
                    
                    h2_mn = [h2_modify(s1m,s2,L),h2_modify(s2,s1m,L)] # s1向右移  H2(s1m,s2,L)
                    h2_nm = [h2_modify(s1,s2m,L),h2_modify(s2m,s1,L)] # s2向右移  H2(s1,s2m,L)
                    
                    if (h2_mn[0]>two_signals_h2[0])&(h2_mn[0]>h2_nm[0]):
                        two_signals_h2[0] = h2_mn[0]
                        two_signals_lag[0] = tagi
                        
                    elif h2_nm[0]>two_signals_h2[0]:
                        two_signals_h2[0] = h2_nm[0]
                        two_signals_lag[0] = -tagi
                        
                    if (h2_mn[1]>two_signals_h2[1])&(h2_mn[1]>h2_nm[0]):
                        two_signals_h2[1] = h2_mn[1]
                        two_signals_lag[1] = -tagi
                        
                    elif h2_nm[1]>two_signals_h2[1]:
                        two_signals_h2[1] = h2_nm[1]
                        two_signals_lag[1] = tagi
                
                signals_h2[si1,si2] = two_signals_h2[0]
                signals_h2[si2,si1] = two_signals_h2[1]
                signals_lag[si1,si2] = two_signals_lag[0]
                signals_lag[si2,si1] = two_signals_lag[1]
                
        h2_value.append(signals_h2)
        lag_value.append(signals_lag)
    
    #print(time_flexity)
    return h2_value,lag_value    

In [176]:
def H2_dichotomy_filter(signals,start_time,duration,sampleFreq,slideWindow=2,step=1,maxlag=0.1,L=7,HP=0,LP=0):
    '''
    版本6：不求平均,lag算法改进,采用二分法计算最大h2，初始采用7点采样（而不是三点）
    signals：原始信号，numpy矩阵类型（二维矩阵）
    slideWindows:滑动窗口
    duration：信号的持续时间
    HP，LP：接收归一化截止频率
    '''
    signals_num = len(signals)
    if signals_num < 2:
        print("Please input at least 2 signals to compute h2!")
        return None,None
    
    h2_num = int((duration-slideWindow-maxlag)//step + 1)
    if h2_num <= 0:
        print("Duration is too short to compute h2!")
        return None,None
    
    if maxlag <= 0:
        print("Time maxlag must be greater than zero!")
    
    b = a = []
    if HP==0 and LP>0: #低通
        b,a = scipy.signal.butter(2,2*LP/sampleFreq,"lowpass")
    elif LP==0 and HP>0: #高通
        b,a = scipy.signal.butter(2,2*HP/sampleFreq,"highpass")
    elif LP>0 and HP>0: #带通
        b,a = scipy.signal.butter(2,[2*HP/sampleFreq,2*LP/sampleFreq],"bandpass")
    
    h2_value = []
    lag_value = []
    dict_sample = dict()
     # 计算每次窗口滑动时的H2（每个时间段）
    for ti in range(h2_num):
        start_time_window = start_time + ti*step
        stop_time_window = start_time_window + slideWindow
        start_samples = int(start_time_window*sampleFreq)
        stop_samples = int(stop_time_window*sampleFreq)
        # 对角线上的值设置为 0
        signals_h2 = np.zeros((signals_num,signals_num))
        signals_lag = np.zeros((signals_num,signals_num))
        # 计算每两个信号之间的H2: (0,1)，(0,2)，...，(0,n-1)；(1,2)，(1,3)，...，(1,n-1)；...(n-2,n-1)
        for si1 in range(signals_num-1):
            s1 = signals[si1][start_samples:stop_samples]
            if len(a):
                s1 = scipy.signal.filtfilt(b,a,s1)
            # [s1] = setSignaltoZero([signals[si1][start_samples:stop_samples]]) # 做归0处理
            
            for si2 in range(si1+1,signals_num):
                s2 = signals[si2][start_samples:stop_samples]
                if len(a):
                    s2 = scipy.signal.filtfilt(b,a,s2)
                #[s2] = setSignaltoZero([signals[si2][start_samples:stop_samples]])
                
                # 设置两个信号之间的H2的初始值为0
                two_signals_h2 = np.zeros(2) 
                two_signals_lag = np.zeros(2) 
                
                h2_00 = [h2_modify(s1,s2,L),h2_modify(s2,s1,L)]
                
                two_signals_h2[0] = h2_00[0]
                two_signals_h2[1] = h2_00[1]
                # 考虑时间延迟
                # 计算x到y
                
                maxlag_points = int(maxlag*sampleFreq) # 总共样本点
                sample_num = int(maxlag_points**0.5-maxlag_points**0.25)
                if sample_num < 3:
                    sample_num = 3
                sample_points = np.percentile(range(0,maxlag_points),np.linspace(0,100,sample_num),interpolation='nearest')
                
                t0_h2 = []
                t1_h2 = []
                dict_sample.clear()
                sign = 1 # 符号位，判断是x移动还是y移动
                left = right = new_left = new_median = new_right = 0
                
                for tagi in sample_points[1:]: # x向右移动，符号为正；y向右移动，符号为负
                    
                    s1m = signals[si1][start_samples+tagi:stop_samples+tagi]
                    s2m = signals[si2][start_samples+tagi:stop_samples+tagi]

                    h2_mn = [h2_modify(s1m,s2,L),h2_modify(s2,s1m,L)] # s1向右移
                    h2_nm = [h2_modify(s1,s2m,L),h2_modify(s2m,s1,L)] # s2向右移

                    t0_h2.extend([h2_mn[0],h2_nm[0]])
                    t1_h2.extend([h2_mn[1],h2_nm[1]])
                
                t0_h2_max = max(t0_h2)
                t1_h2_max = max(t1_h2)
                
                t0_index = t0_h2.index(t0_h2_max)
                t1_index = t1_h2.index(t1_h2_max)
                
                sign = (-1)**t0_index
                if t0_h2_max>two_signals_h2[0]:
                    two_signals_h2[0] = t0_h2_max
                    sub0_h2 = t0_index//2 + 1
                    two_signals_lag[0] = sign*sample_points[sub0_h2]
                    if sub0_h2 < (sample_num - 1):
                        left = sample_points[sub0_h2-1]
                        right = sample_points[sub0_h2+1]
                        new_median = sample_points[sub0_h2]
                        new_left = (left+new_median)//2
                        new_right = (right+new_median)//2
                        #dict_sample[left] = t0_h2[t0_index-2]
                        dict_sample[new_median] = t0_h2[t0_index]
                        #dict_sample[right] = t0_h2[t0_index+2]
                    else:
                        left = sample_points[sub0_h2-2]
                        new_right = right = sample_points[sub0_h2]
                        new_median = (sample_points[sub0_h2]+sample_points[sub0_h2-1])//2
                        new_left = (sample_points[sub0_h2-1]+sample_points[sub0_h2-2])//2
                        dict_sample[sample_points[sub0_h2-1]] = t0_h2[t0_index-2]
                        dict_sample[right] = t0_h2[t0_index]
                else:
                    right = sample_points[2]
                    new_left = left = 0
                    new_right = (right+sample_points[1])//2
                    new_median = sample_points[1]//2
                    dict_sample[sample_points[1]] = t0_h2[(1-sign)//2]
                    dict_sample[0] = two_signals_h2[0]
                
                #print(sign,t0_index,sample_points[t0_index//2 + 1],left,right,new_left,new_median,new_right)
                while (new_right-new_left) >= 2:
                    t0_h2 = []
                    if dict_sample.get(new_left):
                        t0_h2.append(dict_sample[new_left])
                    else:
                        if sign>0: # 说明sx向右移
                            sxx = signals[si1][start_samples+new_left:stop_samples+new_left]
                            t0_h2.append(h2_modify(sxx,s2,L))
                        else:
                            syy = signals[si2][start_samples+new_left:stop_samples+new_left]
                            t0_h2.append(h2_modify(s1,syy,L))
                        dict_sample[new_left] = t0_h2[0]
                            
                    if dict_sample.get(new_median):
                        t0_h2.append(dict_sample[new_median])
                    else:
                        if sign>0: # 说明sx向右移
                            sxx = signals[si1][start_samples+new_median:stop_samples+new_median]
                            t0_h2.append(h2_modify(sxx,s2,L))
                        else:
                            syy = signals[si2][start_samples+new_median:stop_samples+new_median]
                            t0_h2.append(h2_modify(s1,syy,L))
                        dict_sample[new_median] = t0_h2[1]
                        
                    if dict_sample.get(new_right):
                        t0_h2.append(dict_sample[new_right])
                    else:
                        if sign>0: # 说明sx向右移
                            sxx = signals[si1][start_samples+new_right:stop_samples+new_right]
                            t0_h2.append(h2_modify(sxx,s2,L))
                        else:
                            syy = signals[si2][start_samples+new_right:stop_samples+new_right]
                            t0_h2.append(h2_modify(s1,syy,L))
                        dict_sample[new_right] = t0_h2[2]
                            
                    two_signals_h2[0] = max(t0_h2)
                    t0_index = t0_h2.index(two_signals_h2[0])
                    
                    if t0_index == 1:
                        two_signals_lag[0] = sign*new_median
                        left = new_left
                        right = new_right
                        new_left = (new_left+new_median)//2
                        new_right = (new_median+new_right)//2
                    elif t0_index == 0:
                        two_signals_lag[0] = sign*new_left
                        if left==new_left:
                            right = new_right = new_median
                            new_median = (new_right+new_left)//2
                        else:
                            right = new_median
                            new_right = (new_median+new_left)//2
                            new_median = new_left
                            new_left = (left+new_left)//2
                    else:
                        two_signals_lag[0] = sign*new_right
                        if right==new_right:
                            left = new_left = new_median
                            new_median = (new_right+new_left)//2
                        else:
                            left = new_median
                            new_left = (new_median+new_right)//2
                            new_median = new_right
                            new_right = (new_right+right)//2
                    
                sign = (-1)**(t1_index+1)
                dict_sample.clear()
                if t1_h2_max>two_signals_h2[1]:
                    two_signals_h2[1] = t1_h2_max
                    sub1_h2 = t1_index//2 + 1
                    two_signals_lag[1] = sign*sample_points[sub1_h2]
                    if sub1_h2 < (sample_num - 1):
                        left = sample_points[sub1_h2-1]
                        right = sample_points[sub1_h2+1]
                        new_median = sample_points[sub1_h2]
                        new_left = (left+new_median)//2
                        new_right = (right+new_median)//2
                        #dict_sample[left] = t0_h2[t0_index-2]
                        dict_sample[new_median] = t1_h2[t1_index]
                        #dict_sample[right] = t0_h2[t0_index+2]
                    else:
                        left = sample_points[sub1_h2-2]
                        new_right = right = sample_points[sub1_h2]
                        new_median = (sample_points[sub1_h2]+sample_points[sub1_h2-1])//2
                        new_left = (sample_points[sub1_h2-1]+sample_points[sub1_h2-2])//2
                        dict_sample[sample_points[sub1_h2-1]] = t1_h2[t1_index-2]
                        dict_sample[right] = t1_h2[t1_index]
                else:
                    right = sample_points[2]
                    new_left = left = 0
                    new_right = (right+sample_points[1])//2
                    new_median = sample_points[1]//2
                    dict_sample[sample_points[1]] = t1_h2[(1+sign)//2]
                    dict_sample[0] = two_signals_h2[1]
                
                #print(sign,t0_index,sample_points[t0_index//2 + 1],left,right,new_left,new_median,new_right)
                while (new_right-new_left) >= 2:
                    t1_h2 = []
                    if dict_sample.get(new_left):
                        t1_h2.append(dict_sample[new_left])
                    else:
                        if sign<0: # 说明sx向右移
                            sxx = signals[si1][start_samples+new_left:stop_samples+new_left]
                            t1_h2.append(h2_modify(s2,sxx,L))
                        else:
                            syy = signals[si2][start_samples+new_left:stop_samples+new_left]
                            t1_h2.append(h2_modify(syy,s1,L))
                        dict_sample[new_left] = t1_h2[0]
                            
                    if dict_sample.get(new_median):
                        t1_h2.append(dict_sample[new_median])
                    else:
                        if sign<0: # 说明sx向右移
                            sxx = signals[si1][start_samples+new_median:stop_samples+new_median]
                            t1_h2.append(h2_modify(s2,sxx,L))
                        else:
                            syy = signals[si2][start_samples+new_median:stop_samples+new_median]
                            t1_h2.append(h2_modify(syy,s1,L))
                        dict_sample[new_median] = t1_h2[1]
                        
                    if dict_sample.get(new_right):
                        t1_h2.append(dict_sample[new_right])
                    else:
                        if sign<0: # 说明sx向右移
                            sxx = signals[si1][start_samples+new_right:stop_samples+new_right]
                            t1_h2.append(h2_modify(s2,sxx,L))
                        else:
                            syy = signals[si2][start_samples+new_right:stop_samples+new_right]
                            t1_h2.append(h2_modify(syy,s1,L))
                        dict_sample[new_right] = t1_h2[2]
                            
                    two_signals_h2[1] = max(t1_h2)
                    t1_index = t1_h2.index(two_signals_h2[1])
                    
                    if t1_index == 1:
                        two_signals_lag[1] = sign*new_median
                        left = new_left
                        right = new_right
                        new_left = (new_left+new_median)//2
                        new_right = (new_median+new_right)//2
                    elif t1_index == 0:
                        two_signals_lag[1] = sign*new_left
                        if left==new_left:
                            right = new_right = new_median
                            new_median = (new_right+new_left)//2
                        else:
                            right = new_median
                            new_right = (new_median+new_left)//2
                            new_median = new_left
                            new_left = (left+new_left)//2
                    else:
                        two_signals_lag[1] = sign*new_right
                        if right==new_right:
                            left = new_left = new_median
                            new_median = (new_right+new_left)//2
                        else:
                            left = new_median
                            new_left = (new_median+new_right)//2
                            new_median = new_right
                            new_right = (new_right+right)//2
                            
                    #print(t0_index,left,right,new_left,new_median,new_right)        
                #print(dict_sample)
                
                signals_h2[si1,si2] = two_signals_h2[0]
                signals_h2[si2,si1] = two_signals_h2[1]
                signals_lag[si1,si2] = two_signals_lag[0]
                signals_lag[si2,si1] = two_signals_lag[1]
                
        h2_value.append(signals_h2)
        lag_value.append(signals_lag)
    
    #print(time_flexity)
    return h2_value,lag_value

In [169]:
def print_h2_lag(h2_value,lag_value):
    for i,h2_i,lag_i in zip(range(len(h2_value)),h2_value,lag_value):
        print(i+1,"：",lag_i[0],lag_i[1],'\n',h2_i)
        print()

In [109]:
s1=record[56][1000:2000]
s2=record[66][1000:2000]
sxy=pd.DataFrame({'x':s1,'y':s2})
bins=np.linspace(np.min(s1)-0.0001,np.max(s1),7)
sxy['bins']=pd.cut(sxy['x'],bins,labels=range(1,len(bins)),include_lowest=True)

In [163]:
signals1 = np.array([record[56],record[66]])
signals2 = np.array([record[46],record[56]])
signals3 = np.array([record[46],record[56],record[66]])

In [52]:
sxy.iloc[0]['y']

-636.826138704509

In [110]:
bins

array([8408.20508807, 8440.01761205, 8471.83013602, 8503.64266   ,
       8535.45518398, 8567.26770796, 8599.08023194])

In [79]:
t_start = datetime.datetime.now()
h2_11 = h2(record[46][1000:2000],record[56][1000:2000],7)
t_end = datetime.datetime.now()
print(t_end-t_start)

Q_P_k： 0:00:00.010995
计算f： 0:00:00.664755
计算h2： 0:00:00
0:00:00.676749


In [161]:
t_start = datetime.datetime.now()
h2_11 = h2(record[46][1000:2000],record[56][1000:2000],7)
t_end = datetime.datetime.now()
print(t_end-t_start)

Q_P_k： 0:00:00.010996
切割： 0:00:00.002998
计算f： 0:00:00.617773
计算h2： 0:00:00
0:00:00.631767


In [126]:
h2_11

0.060101813316772446

In [159]:
t_start = datetime.datetime.now()
h2_21 = h2_modify(record[46][1000:2000],record[56][1000:2000])
t_end = datetime.datetime.now()
print(t_end-t_start)

0:00:00.006995


In [160]:
h2_21

0.06010181331630571

In [168]:
t_start = datetime.datetime.now()
h2_f1,lag_f1 = H2_filter(signals2,0,10,1000,slideWindow=2,step=1,maxlag=0.1)
t_end = datetime.datetime.now()
print(t_end-t_start)

1593
0:00:00.247909


In [171]:
print_h2_lag(h2_f1,lag_f1)

1 ： [0. 0.] [0. 0.] 
 [[0.         0.01808417]
 [0.02692431 0.        ]]

2 ： [0. 0.] [0. 0.] 
 [[0.         0.32238066]
 [0.31455404 0.        ]]

3 ： [0. 0.] [0. 0.] 
 [[0.         0.17195665]
 [0.21871529 0.        ]]

4 ： [0. 0.] [0. 0.] 
 [[0.         0.0982786 ]
 [0.09627234 0.        ]]

5 ： [0. 0.] [0. 0.] 
 [[0.         0.15186384]
 [0.00982948 0.        ]]

6 ： [0. 0.] [0. 0.] 
 [[0.         0.20296778]
 [0.08522396 0.        ]]

7 ： [0. 0.] [0. 0.] 
 [[0.         0.06732993]
 [0.09029459 0.        ]]

8 ： [0. 0.] [0. 0.] 
 [[0.         0.09893968]
 [0.157991   0.        ]]



In [175]:
t_start = datetime.datetime.now()
h2_f2,lag_f2 = H2_filter(signals2,0,10,1000,slideWindow=2,step=1,maxlag=0.1)
t_end = datetime.datetime.now()
print(t_end-t_start)

1593
0:00:49.389857


In [174]:
print_h2_lag(h2_f2,lag_f2)

1 ： [ 0. 90.] [-90.   0.] 
 [[0.         0.22235251]
 [0.16583793 0.        ]]

2 ： [  0. -50.] [30.  0.] 
 [[0.         0.3877137 ]
 [0.37726124 0.        ]]

3 ： [  0. -90.] [-60.   0.] 
 [[0.         0.31107288]
 [0.31487182 0.        ]]

4 ： [ 0. 80.] [-80.   0.] 
 [[0.         0.16943156]
 [0.20150836 0.        ]]

5 ： [  0. -20.] [90.  0.] 
 [[0.         0.17348006]
 [0.0403989  0.        ]]

6 ： [ 0. 40.] [90.  0.] 
 [[0.         0.22538822]
 [0.11393726 0.        ]]

7 ： [ 0. 99.] [-16.   0.] 
 [[0.         0.10700302]
 [0.09872842 0.        ]]

8 ： [ 0. 40.] [-40.   0.] 
 [[0.         0.11028308]
 [0.17347399 0.        ]]



In [183]:
t_start = datetime.datetime.now()
h2_df1,lag_df1 = H2_dichotomy_filter(signals2,0,10,1000,slideWindow=2,step=1,maxlag=0.1)
t_end = datetime.datetime.now()
print(t_end-t_start)

0:00:04.850217


In [184]:
print_h2_lag(h2_df1,lag_df1)

1 ： [ 0. 90.] [-90.   0.] 
 [[0.         0.22235251]
 [0.16583793 0.        ]]

2 ： [  0. -46.] [30.  0.] 
 [[0.         0.38352   ]
 [0.37726124 0.        ]]

3 ： [  0. -90.] [-60.   0.] 
 [[0.         0.31107288]
 [0.31487182 0.        ]]

4 ： [  0. -99.] [-99.   0.] 
 [[0.         0.16592513]
 [0.19340851 0.        ]]

5 ： [  0. -20.] [-70.   0.] 
 [[0.         0.17348006]
 [0.03888954 0.        ]]

6 ： [ 0. 40.] [90.  0.] 
 [[0.         0.22538822]
 [0.11393726 0.        ]]

7 ： [ 0. 99.] [-16.   0.] 
 [[0.         0.10700302]
 [0.09872842 0.        ]]

8 ： [ 0. 40.] [-40.   0.] 
 [[0.         0.11028308]
 [0.17347399 0.        ]]



In [190]:
h2_f3,lag_f3 = H2_filter(signals2,0,10,1000,slideWindow=2,step=1,maxlag=0.1,HP=1,LP=45)

In [187]:
print_h2_lag(h2_f3,lag_f3)

1 ： [ 0. 90.] [22.  0.] 
 [[0.         0.13544694]
 [0.09953406 0.        ]]

2 ： [  0. -45.] [34.  0.] 
 [[0.         0.19309163]
 [0.19213968 0.        ]]

3 ： [  0. -99.] [89.  0.] 
 [[0.         0.09243174]
 [0.14953547 0.        ]]

4 ： [  0. -92.] [-81.   0.] 
 [[0.         0.16208628]
 [0.14914188 0.        ]]

5 ： [ 0. -1.] [80.  0.] 
 [[0.         0.21224279]
 [0.07859549 0.        ]]

6 ： [ 0. 81.] [95.  0.] 
 [[0.         0.20841801]
 [0.14558748 0.        ]]

7 ： [0. 2.] [4. 0.] 
 [[0.         0.14786378]
 [0.20623865 0.        ]]

8 ： [0. 0.] [0. 0.] 
 [[0.         0.26385045]
 [0.21083102 0.        ]]



In [191]:
t_start = datetime.datetime.now()
h2_df3,lag_df3 = H2_filter(signals2,0,10,1000,slideWindow=2,step=1,maxlag=0.2)
t_end = datetime.datetime.now()
print(t_end-t_start)

0:01:51.276130


In [193]:
print_h2_lag(h2_df3,lag_df3)

1 ： [   0. -170.] [170.   0.] 
 [[0.         0.32315543]
 [0.33494444 0.        ]]

2 ： [  0. -50.] [30.  0.] 
 [[0.         0.3877137 ]
 [0.37726124 0.        ]]

3 ： [  0. -90.] [130.   0.] 
 [[0.         0.31107288]
 [0.34903774 0.        ]]

4 ： [   0. -110.] [-100.    0.] 
 [[0.         0.19811423]
 [0.2063928  0.        ]]

5 ： [  0. -20.] [190.   0.] 
 [[0.         0.17348006]
 [0.14632529 0.        ]]

6 ： [ 0. 40.] [110.   0.] 
 [[0.         0.22538822]
 [0.12013521 0.        ]]

7 ： [  0. 110.] [199.   0.] 
 [[0.         0.12606271]
 [0.10691637 0.        ]]

8 ： [  0. 199.] [-199.    0.] 
 [[0.         0.19443809]
 [0.22594219 0.        ]]

